In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
ROI = ee.FeatureCollection('users/311605001111/YangtzeCity/chongqing').geometry()
Map.addLayer(ROI, {}, "roi")
Map.centerObject(ROI,6)

In [ ]:
# # Region of Interest
# ROI = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').geometry()
# lon= ee.Number(ee.Feature(ROI.centroid(100)).geometry().coordinates().get(0))
# lat= ee.Number(ee.Feature(ROI.centroid(100)).geometry().coordinates().get(1))
# print(lon.getInfo(), lat.getInfo())

In [4]:
# Permanent Waterbody Mask
perm = ee.Image('JRC/GSW1_2/GlobalSurfaceWater').select('transition')
perm = perm.updateMask(perm.eq(1))
perm = perm.reduceToVectors(**{
    'geometry': ROI, 
    'geometryType': 'polygon',
    'scale': 100,
    'maxPixels': 1e8
})
Map.addLayer(perm, {}, "JRC perm")

In [ ]:
## Global variables

# SAR
Sen1 = 'COPERNICUS/S1_GRD'
pol = 'VV'
instr = 'IW'
res = 10

# Landsat
LS8 = 'LANDSAT/LC08/C01/T1_SR'
LS7 = 'LANDSAT/LE07/C01/T1_SR'
# Number of cluster
clnum = 5

# Color Palettes for Visualization
vis_MNDWI = {
    'palette': ['00FFFF', '0000FF'],
    'min': [-1],
    'max': [1]
}

vis_SAR = {
    'palette': ['#3f007d','#efedf5'],
    'min': [-20],
    'max': [-5]
}

vis_WB = {
    'min':0,
    'max':1,
    'palette': ['#fff7f3', '#dd3497']
}


In [ ]:
##  SAR Functions

# Function to mask border noise
def maskEdge(img):
    clipped = img.clip(img.geometry().buffer(-5000))
    mask = img.select(0).unitScale(-25, 5).multiply(255).toByte().connectedComponents(ee.Kernel.rectangle(1,1), 256)
    masked = img.updateMask(mask.select(0)).unmask(clipped,False)
    mask2 = masked.mask()
    mask3 = mask2.focal_min(20, 'plus', 'meters')
    return img.updateMask(mask3)

# Function to generate SAR median image (+/-15 days)
def SAR_process(Sen1, d, ROI):
    date = d
    start = ee.Date.parse('yyyy-MM-dd', d).advance(-15, 'day')
    end = ee.Date.parse('yyyy-MM-dd', d).advance(15, 'day')
    SAR = ee.ImageCollection(Sen1) \
        .filterBounds(ROI) \
        .filterDate(start, end) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol)) \
        .filter(ee.Filter.eq('instrumentMode', instr)) \
        .filter(ee.Filter.eq('resolution_meters', res)) \
        .map(maskEdge) \
        .select('VV')
    return SAR.median()

In [ ]:
## Landsat Functions

# Function to mask cloud pixels.
def maskIm(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

# Functions to calculate MNDWI
def DI_LS8(image):
    index = image.normalizedDifference(['B3','B6']).rename('MNDWI')
    return image.addBands(index)

def DI_LS7(image):
    index = image.normalizedDifference(['B2','B5']).rename('MNDWI')
    return image.addBands(index)

# Function to calculate LS median
def LSMed(d, ROI, days):
    start = ee.Date(d).advance(ee.Number(days).multiply(-1), 'day')
    end = ee.Date(d).advance(ee.Number(days), 'day')
    LS8Col = ee.ImageCollection(LS8) \
        .filterBounds(ROI) \
        .filterDate(start, end) \
        .map(maskIm) \
        .map(DI_LS8) \
        .select(['MNDWI']) 
    LS7Col = ee.ImageCollection(LS7) \
        .filterBounds(ROI) \
        .filterDate(start, end) \
        .map(maskIm) \
        .map(DI_LS7) \
        .select(['MNDWI'])
    LS = LS8Col.merge(LS7Col)
    return ee.ImageCollection(LS).median()

# Function to generate a gap-filled MNDWI image 
# Fill image is the median of the median images for the same month 2007-2016
def landsat_process(d, ROI):
    date = ee.Date.parse('yyyy-MM-dd', d)
    month = date.get('month')
    yrs = ee.List.sequence(2007,2016)
    dates = yrs.map(lambda y : ee.Date.fromYMD(ee.Number(y),month,15))
    col = dates.map(lambda d : LSMed(d, ROI, 15))
    fill = ee.ImageCollection(col).median()
    LS = ee.Image(LSMed(date, ROI, 15)).unmask(fill)
    col = dates.map(lambda d : LSMed(d, ROI, 45))
    fill = ee.ImageCollection(col).median()
    LS = ee.Image(LS.unmask(fill))
    return ee.Image(LS)

In [ ]:
# image = landsat_process('2016-06-15',ROI).clip(ROI)
# Map.addLayer(image.select(['MNDWI']), vis_MNDWI, 'MNDWI Composite')
# d = '2016-06-15'
# SAR = SAR_process(Sen1, d, ROI)
# LS = landsat_process(d, ROI)
# img = SAR.addBands(LS.select(['MNDWI'])).clip(ROI)
# Map.addLayer(img.select(['VV']),  vis_SAR, 'SAR Composite')

In [ ]:
## Cluster Functions

# Function to make training set
def train(img,clnum,clusArea):
    image = img.clip(clusArea)
    min = clnum
    max = clnum
    training_features = image.sample(**{
        'region': clusArea,
        'scale': 10,
        'numPixels': 1e3
    })
    clusterer = ee.Clusterer.wekaCascadeKMeans(**{
        'minClusters' : min,
        'maxClusters': max,
        'restarts': 10,
        'distanceFunction': 'Euclidean',
        'maxIterations' : 6
    })
    return clusterer.train(training_features)

# Function to perform cluster analysis
def clusSAR_LS(img, clusterer, bound):
    image = img.clip(bound)
    kmeans_image = image.cluster(clusterer)
    return kmeans_image

# Function to get water cluster
def water(kmeans_image,lake):
    limit = 1000
    WB = ee.Image(kmeans_image.select(['cluster']))
    clNumber = WB.sampleRegions(**{
        'collection': lake, 
        'scale': 10,
    })
    clNumber = clNumber.limit(limit) \
        .toList(limit) \
        .map(lambda f : ee.Feature(f).get('cluster')) \
        .reduce(ee.Reducer.mode()) 
    WB = WB.eq(ee.Number(clNumber))
    return WB.updateMask(WB.eq(1)) 

In [ ]:
## Master Function
def getWB(d, clnum, bound, lake):
    SAR = SAR_process(Sen1, d, bound)
    LS = landsat_process(d, bound)
    img = SAR.addBands(LS.select(['MNDWI']))
    clusterer = train(img, clnum, bound)
    kmeans_image = clusSAR_LS(img, clusterer, bound)
    wb = water(kmeans_image, lake)
    kmeans_image = kmeans_image.add(ee.Number(1))
    kmeans_image = kmeans_image.int()
    Map.addLayer(SAR.clip(ROI), vis_SAR, 'SAR Composite')
    Map.addLayer(LS.select(['MNDWI']).clip(ROI), vis_MNDWI, 'MNDWI Composite')
    Map.addLayer(ee.Image(kmeans_image).randomVisualizer(), {}, 'Kmeans Clusters')
    Map.addLayer(wb, vis_WB, 'Surface Water')
    return wb

In [ ]:
# Run Script

date = '2019-07-15'
getWater = getWB(date, clnum, ROI, perm)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

# region = [105.5, 28.5, 110.2, 31.5] 
region = [106, 29, 107, 30.5] 
vis = {'bands': ['cluster'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(getWater, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'APWC 2019/07', fontsize=20)